## Import and Load

In [96]:
#import tools, load required data
import sys
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import re
import csv
import os
pd.set_option("display.max_rows",300)
pd.set_option("display.max_columns",100)

In [97]:
#load all files
preDX = pd.read_csv("/home/ucsdantibiogram/notebooks/data/round2data/Diag_before_order.csv", header = None, names = ("MRN", "Visit_id", "ICD9", "ICD10", "preDX_time", "PP_YN"))
orderDX = pd.read_csv("/home/ucsdantibiogram/notebooks/data/round2data/Diag_at_order.csv", header = None, names = ("MRN", "Visit_id", "ICD9", "ICD10", "preDX_time", "PP_YN"))
medsBeforeOrder = pd.read_csv("/home/ucsdantibiogram/notebooks/data/round2data/Med_before_order.csv", header = None, names = ("MRN", "Visit_id", "Order_id", "Med_name", "order_mode", "Patient_type", "ordering_date", "start_date"))
medsAtOrder = pd.read_csv("/home/ucsdantibiogram/notebooks/data/round2data/Med_at_order.csv", header = None, names = ("MRN", "Visit_id", "Order_id", "Med_name", "order_mode", "Patient_type", "ordering_date", "start_date"))
medsAfterOrder = pd.read_csv("/home/ucsdantibiogram/notebooks/data/round2data/Med_after_order.csv", header = None, names = ("MRN", "Visit_id", "Order_id", "Med_name", "order_mode", "Patient_type", "ordering_date", "start_date"))
orderInfo = pd.read_csv("/home/ucsdantibiogram/notebooks/data/round2data/Order_info.csv", header = None, names = ("MRN", "Visit_id", "Order_id", "Order_time", "line", "description", "antibiotic", "susceptibility", "sensitivity_value", "organism", "specimen_taken_time", "specimen_comments", "specimen_type", "result_date", "hosp_dept", "enc_dept", "admission_time", "specimen_source", "Discharge_Disposition", "Discharge_time"))
problem_list = pd.read_csv("/home/ucsdantibiogram/notebooks/data/round2data/Problem_List.csv", header = None, names = ("MRN", "Visit_id", "ICD9", "ICD10", "DX_time", "PP_YN", "Activity"))
demo = pd.read_csv("/home/ucsdantibiogram/notebooks/data/round2data/Demographic.csv")
social = pd.read_csv("/home/ucsdantibiogram/notebooks/data/round2data/social.csv")
dischargeDiag = pd.read_csv("/home/ucsdantibiogram/notebooks/data/round2data/discharge_diag.csv")
dischargeMeds = pd.read_csv("/home/ucsdantibiogram/notebooks/data/round2data/discharge_meds.csv", encoding = "ISO-8859-1")

# Another try

In [212]:
antibiotic_bugs_susc = orderInfo[['antibiotic','organism','susceptibility']]

In [213]:
antibiotic_bugs_susc['organism'].unique()

array(['ESCHERICHIA COLI', 'STAPHYLOCOCCUS, COAGULASE NEGATIVE',
       'STREPTOCOCCUS, VIRIDANS GROUP',
       'METHICILLIN RESISTANT STAPHYLOCOCCUS AUREUS',
       'STAPHYLOCOCCUS AUREUS', 'STENOTROPHOMONAS MALTOPHILIA',
       'ENTEROBACTER CLOACAE', 'PSEUDOMONAS AERUGINOSA',
       'ALCALIGENES FAECALIS', 'ENTEROCOCCUS FAECALIS',
       'PROTEUS MIRABILIS', 'SERRATIA MARCESCENS', 'CANDIDA ALBICANS',
       'VANCOMYCIN RESISTANT ENTEROCOCCUS FAECIUM', 'ALCALIGENES SPECIES',
       'KLEBSIELLA PNEUMONIAE SS. PNEUMONIAE',
       'ACHROMOBACTER DENITRIFICANS', 'ENTEROBACTER SPECIES',
       'SERRATIA SPECIES', 'PROVIDENCIA RETTGERI', 'MYROIDES ODORATIMIMUS',
       'ESBL ESCHERICHIA COLI',
       'MYCOBACTERIUM AVIUM-INTRACELLULARE COMPLEX', 'PROTEUS VULGARIS',
       'MORGANELLA MORGANII', 'KLEBSIELLA OXYTOCA',
       'ESBL KLEBSIELLA PNEUMONIAE', 'STAPHYLOCOCCUS SPECIES',
       'STAPHYLOCOCCUS EPIDERMIDIS', 'STAPHYLOCOCCUS HAEMOLYTICUS',
       'ENTEROBACTER GERGOVIAE', 'PROTEUS VUL

In [214]:
# Drop all duplicates
antibiotic_bugs_susc_noduplicates = antibiotic_bugs_susc.drop_duplicates()

In [215]:
# Group by antibiotic, find all organism and apply the aggregate function
result = antibiotic_bugs_susc_noduplicates.groupby(['antibiotic'])['organism'].nunique()

In [216]:
antibiotic_bugs_susc['susceptibility'] = antibiotic_bugs_susc['susceptibility'].map(lambda x: 1 if x == 'Susceptible' else (0.5 if x == 'Intermediate' else 0))

/home/cuda/anaconda3/envs/py3/lib/python3.5/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [217]:
antibiotic_bugs_susc

,antibiotic,organism,susceptibility
0,Amikacin,ESCHERICHIA COLI,1.0
1,Ampicillin,ESCHERICHIA COLI,1.0
2,Ampicillin/Sulbactam,ESCHERICHIA COLI,1.0
3,Cefazolin,ESCHERICHIA COLI,1.0
4,Cefepime,ESCHERICHIA COLI,1.0
5,Cefoxitin,ESCHERICHIA COLI,1.0
6,Ceftazidime,ESCHERICHIA COLI,1.0
7,Ceftriaxone,ESCHERICHIA COLI,1.0
8,Cefuroxime,ESCHERICHIA COLI,1.0
9,Ciprofloxacin,ESCHERICHIA COLI,1.0


In [218]:
# How many test against
count = antibiotic_bugs_susc.groupby(['antibiotic'])['susceptibility'].count()
# Susceptible will count as 1, intermediate 0.5, others 0
susc_sum = antibiotic_bugs_susc.groupby(['antibiotic'])['susceptibility'].sum()

In [219]:
count = count.rename("count")
susc_sum = susc_sum.rename("sum")
new_df = pd.concat([result, count, susc_sum], axis=1)

In [220]:
new_df['percentage'] = new_df['sum']/new_df['count']

In [221]:
new_df = new_df[['organism','percentage']]

In [222]:
new_df

,organism,percentage
antibiotic,,
5-Flucytosine,9,0.191176
Amikacin,205,0.941538
Amoxicillin,5,0.950000
Amoxicillin/Clavulanate,12,0.437500
Amphotericin-B,29,0.062937
Ampicillin,166,0.370416
Ampicillin/Sulbactam,134,0.539862
Anidulafungin,1,1.000000
Azithromycin,23,0.614525


## Organism vs Times Tested vs %Susceptible

In [95]:
antiList = orderInfo[['antibiotic', 'organism', 'susceptibility']]
antiList.drop_duplicates(inplace = True)

antiList['anticount'] = antiList.groupby('antibiotic')['organism'].nunique()

antiList

/home/cuda/anaconda3/envs/py3/lib/python3.5/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/home/cuda/anaconda3/envs/py3/lib/python3.5/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,antibiotic,organism,susceptibility,anticount
0,Amikacin,ESCHERICHIA COLI,Susceptible,NaN
1,Ampicillin,ESCHERICHIA COLI,Susceptible,NaN
2,Ampicillin/Sulbactam,ESCHERICHIA COLI,Susceptible,NaN
3,Cefazolin,ESCHERICHIA COLI,Susceptible,NaN
4,Cefepime,ESCHERICHIA COLI,Susceptible,NaN
5,Cefoxitin,ESCHERICHIA COLI,Susceptible,NaN
6,Ceftazidime,ESCHERICHIA COLI,Susceptible,NaN
7,Ceftriaxone,ESCHERICHIA COLI,Susceptible,NaN
8,Cefuroxime,ESCHERICHIA COLI,Susceptible,NaN
9,Ciprofloxacin,ESCHERICHIA COLI,Susceptible,NaN


## Within Visit ID, keep visits within a week apart

In [236]:
allVisits

,MRN,Visit_id,Order_time,time_shift
336267,16267767,16244531,2011-11-16 00:00:00.000,NaN
530297,21623236,16303797,2011-12-02 16:57:00.000,NaN
397158,18042440,16339907,2011-12-06 10:45:00.000,NaN
66388,6629406,16447375,2011-12-12 14:52:00.000,NaN
129371,8839961,16586417,2012-01-13 09:33:00.000,NaN
531644,21660253,16624748,2012-01-18 13:23:00.000,NaN
46773,5904040,16628017,2011-12-20 00:00:00.000,NaN
654684,24365793,16634878,2012-01-11 14:55:00.000,NaN
558294,22306641,16659855,2011-11-22 13:22:00.000,NaN
666979,24592206,16719525,2011-12-12 13:58:00.000,NaN


In [234]:
allVisits = orderInfo[['MRN', 'Visit_id', 'Order_time']].drop_duplicates()
allVisits.set_index('Order_time')

allVisits.sort_values(['Visit_id', 'Order_time'], ascending = True, inplace = True)

visitGroups = allVisits.groupby('MRN')

In [235]:
allVisits['time_shift'] = visitGroups['Order_time'].transform(lambda x: x.shift())

pd.to_datetime( allMultipleVisits['Order_time'] )

allVisits['time_diff'] = (allVisits['time_shift'] - allVisits['Order_time']).dt.days

allVisits['keep_idx'] = visitGroups['time_diff'].transform(lambda x: abs(x) <= 7 ) 
# we need to undo the shift we performed before and make sure that
# we capture both records involved. Hence the OR operation.
#allVisits['keep_idx'] = allVisits['keep_idx'] | allVisits.groupby('MRN', 'Visit_id')['keep_idx'].transform(lambda x: x.shift(-1))

#allVisits.loc[allVisits['keep_idx'] > 0] 
allVisits

ValueError: month must be in 1..12

## Find Prescribed Resistant Antibiotics

In [3]:
#Merge limited OrderInfo, MedsAtOrder and filter for Antibiotics

OILimited = orderInfo[["MRN", "Visit_id", "Order_id", "description", "antibiotic", "susceptibility", "organism"]]

antiSet = orderInfo["antibiotic"].str.upper().unique()
MAOLimited = medsAtOrder[["MRN", "Visit_id", "Order_id", "Med_name", "Patient_type"]].copy()
MAOLimited['Med_name']=MAOLimited['Med_name'].str.split().str[0].replace('-', '/')
MAOLimited = MAOLimited[MAOLimited["Med_name"].isin( antiSet )]   
MAOLimited.rename( columns = {"Med_name": "Antis"}, inplace = True)
MAOLimited.drop_duplicates( subset = ("Visit_id", "Antis"), inplace = True)

mergedDF = OILimited.merge( MAOLimited, how = "inner", on = ("MRN", "Visit_id") )
mergedDF

,MRN,Visit_id,Order_id_x,description,antibiotic,susceptibility,organism,Order_id_y,Antis,Patient_type
0,3527611,22257231,66201425,"BODY SITE W/GRAM STAIN, AEROBIC ONLY",Cefazolin,Susceptible,"STAPHYLOCOCCUS, COAGULASE NEGATIVE",66155454,VANCOMYCIN,Inpatient
1,3527611,22257231,66201425,"BODY SITE W/GRAM STAIN, AEROBIC ONLY",Daptomycin,Susceptible,"STAPHYLOCOCCUS, COAGULASE NEGATIVE",66155454,VANCOMYCIN,Inpatient
2,3527611,22257231,66201425,"BODY SITE W/GRAM STAIN, AEROBIC ONLY",Linezolid,Susceptible,"STAPHYLOCOCCUS, COAGULASE NEGATIVE",66155454,VANCOMYCIN,Inpatient
3,3527611,22257231,66201425,"BODY SITE W/GRAM STAIN, AEROBIC ONLY",Oxacillin,Susceptible,"STAPHYLOCOCCUS, COAGULASE NEGATIVE",66155454,VANCOMYCIN,Inpatient
4,3527611,22257231,66201425,"BODY SITE W/GRAM STAIN, AEROBIC ONLY",Rifampin,Susceptible,"STAPHYLOCOCCUS, COAGULASE NEGATIVE",66155454,VANCOMYCIN,Inpatient
5,3527611,22257231,66201425,"BODY SITE W/GRAM STAIN, AEROBIC ONLY",Tetracycline,Susceptible,"STAPHYLOCOCCUS, COAGULASE NEGATIVE",66155454,VANCOMYCIN,Inpatient
6,3527611,22257231,66201425,"BODY SITE W/GRAM STAIN, AEROBIC ONLY",Vancomycin,Susceptible,"STAPHYLOCOCCUS, COAGULASE NEGATIVE",66155454,VANCOMYCIN,Inpatient
7,3527611,22257231,66201425,"BODY SITE W/GRAM STAIN, AEROBIC ONLY",Penicillin G,Susceptible,"STREPTOCOCCUS, VIRIDANS GROUP",66155454,VANCOMYCIN,Inpatient
8,3527611,22257231,66201425,"BODY SITE W/GRAM STAIN, AEROBIC ONLY",Ceftriaxone,Susceptible,"STREPTOCOCCUS, VIRIDANS GROUP",66155454,VANCOMYCIN,Inpatient
9,3527835,19655474,52542155,FLUID CULTURE IN BLOOD BOTTLES,Amikacin,Susceptible,ESCHERICHIA COLI,52542176,CEFOTAXIME,Inpatient


### Filter for Resistant Tests

In [4]:
orderTests = mergedDF[mergedDF['Antis'] == mergedDF['antibiotic'].str.upper()]
orderWithFailedTest = orderTests[orderTests['susceptibility'] == 'Resistant']
orderWithFailedTest.drop_duplicates( subset = "Visit_id", inplace = True )
orderWithFailedTest

/home/cuda/anaconda3/envs/py3/lib/python3.5/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


,MRN,Visit_id,Order_id_x,description,antibiotic,susceptibility,organism,Order_id_y,Antis,Patient_type
121,3530607,17948957,44503940,RESPIRATORY CULTURE W/GRAM STAIN,Ceftazidime,Resistant,STENOTROPHOMONAS MALTOPHILIA,43976607,CEFTAZIDIME,Inpatient
549,3531860,18847426,48153037,"BODY SITE W/GRAM STAIN, AEROBIC ONLY",Ceftazidime,Resistant,ALCALIGENES FAECALIS,48722058,CEFTAZIDIME,Outpatient
589,3531860,19116412,49948183,QUANT TISSUE CULTURE W/GRAM STAIN,Ceftazidime,Resistant,PSEUDOMONAS AERUGINOSA,49945185,CEFTAZIDIME,Inpatient
687,3531860,66005815395,98496578,URINE CULTURE,Ampicillin,Resistant,VANCOMYCIN RESISTANT ENTEROCOCCUS FAECIUM,98702170,AMPICILLIN,Inpatient
1650,3545753,66011833622,124868035,URINE CULTURE,Ciprofloxacin,Resistant,ESCHERICHIA COLI,124868039,CIPROFLOXACIN,Outpatient
1900,3554607,66006391927,99816186,BLOOD CULTURE ROUTINE,Ciprofloxacin,Resistant,ESBL ESCHERICHIA COLI,99816178,CIPROFLOXACIN,Inpatient
2053,3561719,66011965584,126878226,URINE CULTURE,Ciprofloxacin,Resistant,ESBL ESCHERICHIA COLI,126981230,CIPROFLOXACIN,Inpatient
2184,3563616,19512490,52042423,"BODY SITE W/GRAM STAIN, AEROBIC ONLY",Oxacillin,Resistant,METHICILLIN RESISTANT STAPHYLOCOCCUS AUREUS,52076668,OXACILLIN,Outpatient
2413,3570249,66002399386,80975485,URINE CULTURE,Ciprofloxacin,Resistant,MORGANELLA MORGANII,80975488,CIPROFLOXACIN,Inpatient
2645,3577467,66006310736,99426588,ROUTINE COLLECT RESPIRATORY CULTURE W/GRAM STAIN,Cefazolin,Resistant,SERRATIA MARCESCENS,99360670,CEFAZOLIN,Inpatient


In [5]:
#Visits where resistant antibiotics were prescribed / Total amount of visits recorded
len(orderWithFailedTest["Visit_id"].unique())  / len( orderInfo['Visit_id'].unique() )

0.10006220264613419

In [6]:
#Average Visits per MRN
len( orderInfo['Visit_id'].unique()) / len( orderInfo['MRN'].unique())

1.8867918543424476

## How many misprescriptions led to dead patients?

In [7]:
antiToDeath = orderWithFailedTest.copy( deep = True )
demoLimited = demo[["MRN", "death_within_4week", "age_at_order"]]
antiToDeath = antiToDeath.merge( demoLimited, on = ("MRN"), how = "inner")
antiToDeath = antiToDeath[antiToDeath['death_within_4week'] == "Y"].drop_duplicates( subset = "MRN")
antiToDeath

,MRN,Visit_id,Order_id_x,description,antibiotic,susceptibility,organism,Order_id_y,Antis,Patient_type,death_within_4week,age_at_order
1,3530607,17948957,44503940,RESPIRATORY CULTURE W/GRAM STAIN,Ceftazidime,Resistant,STENOTROPHOMONAS MALTOPHILIA,43976607,CEFTAZIDIME,Inpatient,Y,86
19,3563616,19512490,52042423,"BODY SITE W/GRAM STAIN, AEROBIC ONLY",Oxacillin,Resistant,METHICILLIN RESISTANT STAPHYLOCOCCUS AUREUS,52076668,OXACILLIN,Outpatient,Y,58
21,3570249,66002399386,80975485,URINE CULTURE,Ciprofloxacin,Resistant,MORGANELLA MORGANII,80975488,CIPROFLOXACIN,Inpatient,Y,73
95,3725454,66008637580,110717999,"BODY SITE W/GRAM STAIN, AEROBIC ONLY",Vancomycin,Resistant,VANCOMYCIN RESISTANT ENTEROCOCCUS FAECIUM,110691244,VANCOMYCIN,Inpatient,Y,50
100,3738903,18044113,43990546,URINE CULTURE,Meropenem,Resistant,PSEUDOMONAS AERUGINOSA,44044025,MEROPENEM,Inpatient,Y,80
130,3824661,20838460,59071118,ROUTINE COLLECT RESPIRATORY CULTURE W/GRAM STAIN,Ciprofloxacin,Resistant,PSEUDOMONAS AERUGINOSA,59071119,CIPROFLOXACIN,Inpatient,Y,71
137,3844024,19996695,54543537,URINE CULTURE,Ampicillin,Resistant,VANCOMYCIN RESISTANT ENTEROCOCCUS FAECIUM,55044650,AMPICILLIN,Inpatient,Y,46
138,3890076,66006105201,99237079,URINE CULTURE,Ceftazidime,Resistant,CITROBACTER FREUNDII,98423266,CEFTAZIDIME,Inpatient,Y,63
140,3907441,66009858374,116553805,ROUTINE COLLECT RESPIRATORY CULTURE W/GRAM STAIN,Ciprofloxacin,Resistant,PSEUDOMONAS AERUGINOSA,116659840,CIPROFLOXACIN,Inpatient,Y,63
197,4040929,22319701,66447373,URINE CULTURE,Vancomycin,Resistant,VANCOMYCIN RESISTANT ENTEROCOCCUS FAECIUM,66429704,VANCOMYCIN,Inpatient,Y,85


In [8]:
# (#patients ended up dead after one of their visits had a misprescribed anti) / (total patients recorded) 
len( antiToDeath['MRN'].unique() ) / len( orderInfo['MRN'].unique())

0.017921715409503266

In [9]:
# (#patients ended up dead after one of their visits had a misprescribed anti) / (# of misprescriptions given)
565/5952

0.0949260752688172

In [10]:
#sepsisToDeath = sepsis_list[sepsis_list['Visit_id'].isin(antiToDeath['Visit_id'].unique())]
#len( sepsisToDeath['Visit_id'].unique())

NameError: name 'sepsis_list' is not defined

## ICD Codes of Dead Patients

In [11]:
OILimited2 = orderInfo[["MRN", "Visit_id", "Order_id", "Order_time", "description", "antibiotic", "susceptibility", "organism"]]
probListLimited = problem_list[['MRN', 'Visit_id', 'ICD10', 'PP_YN']]
probListLimited = probListLimited[probListLimited['PP_YN'] == 'Y']
probWithTests = OILimited2.merge( probListLimited, on=('MRN', 'Visit_id'), how = 'inner')
demoLimited = demo[['MRN', 'death_within_4week']]
probTestsDeath = probWithTests.merge( demoLimited, on = 'MRN', how = 'inner')
probTestsDeath.drop_duplicates( subset=('MRN', 'Visit_id', 'ICD10'), keep = 'last')

probTestsDeath['ICD10'] = probTestsDeath['ICD10'].str.split('.').str[0]

probTestsDeath

,MRN,Visit_id,Order_id,Order_time,description,antibiotic,susceptibility,organism,ICD10,PP_YN,death_within_4week
0,3531860,17484203,41020026,2011-11-27 00:00:00.000,"BODY SITE W/GRAM STAIN, AEROBIC ONLY",Amikacin,Susceptible,ESCHERICHIA COLI,L97,Y,N
1,3531860,17484203,41020026,2011-11-27 00:00:00.000,"BODY SITE W/GRAM STAIN, AEROBIC ONLY",Amikacin,Susceptible,ESCHERICHIA COLI,L97,Y,N
2,3531860,17484203,41020026,2011-11-27 00:00:00.000,"BODY SITE W/GRAM STAIN, AEROBIC ONLY",Amikacin,Susceptible,ESCHERICHIA COLI,L97,Y,N
3,3531860,17484203,41020026,2011-11-27 00:00:00.000,"BODY SITE W/GRAM STAIN, AEROBIC ONLY",Amikacin,Susceptible,ESCHERICHIA COLI,L97,Y,N
4,3531860,17484203,41020026,2011-11-27 00:00:00.000,"BODY SITE W/GRAM STAIN, AEROBIC ONLY",Amikacin,Susceptible,ESCHERICHIA COLI,A04,Y,N
5,3531860,17484203,41020026,2011-11-27 00:00:00.000,"BODY SITE W/GRAM STAIN, AEROBIC ONLY",Amikacin,Susceptible,ESCHERICHIA COLI,A04,Y,N
6,3531860,17484203,41020026,2011-11-27 00:00:00.000,"BODY SITE W/GRAM STAIN, AEROBIC ONLY",Amikacin,Susceptible,ESCHERICHIA COLI,A04,Y,N
7,3531860,17484203,41020026,2011-11-27 00:00:00.000,"BODY SITE W/GRAM STAIN, AEROBIC ONLY",Amikacin,Susceptible,ESCHERICHIA COLI,A04,Y,N
8,3531860,17484203,41020026,2011-11-27 00:00:00.000,"BODY SITE W/GRAM STAIN, AEROBIC ONLY",Amikacin,Susceptible,ESCHERICHIA COLI,I83,Y,N
9,3531860,17484203,41020026,2011-11-27 00:00:00.000,"BODY SITE W/GRAM STAIN, AEROBIC ONLY",Amikacin,Susceptible,ESCHERICHIA COLI,I83,Y,N


In [12]:
deathWithICD = probTestsDeath[probTestsDeath['death_within_4week'] == 'Y']
deadICDCodesCount = deathWithICD['ICD10'].value_counts()
deadICDCodesCount

A41        26716
R65        12599
N39         6399
R78         3590
E87         2605
I95         2348
J96         2348
E84         2313
J18         1861
I50         1705
K56         1431
R11         1407
N17         1390
R41         1258
R50         1217
K92         1213
D70         1144
N13         1067
K72         1065
K59         1015
R19          938
T81          937
R04          915
J15          897
K65          860
R10          818
N18          730
I62          661
K42          631
D58          619
C78          608
R09          605
T82          579
S72          560
Z91          551
K83          515
J69          449
P07          446
Z95          438
R62          432
R07          419
M72          417
I21          404
R06          399
C79          387
S06          384
A04          371
G89          350
I34          340
R55          331
N10          313
R60          309
I63          309
C80          304
C71          300
D62          299
T21          298
L03          287
A40          2

In [13]:
len( deathWithICD['MRN'].unique() )

628

## Some Numbers 

In [14]:
len( problem_list['MRN'].unique() )

28681

In [15]:
len( orderInfo['MRN'].unique() )

31526

In [16]:
len( demo[demo['death_within_4week'] == 'Y'])

1679

In [17]:
len( social['MRN'].unique() )

29602

In [18]:
159/31526

0.00504345619488676

In [20]:
159/565

0.2814159292035398

## Patient Visit Count

In [66]:
OISmall = orderInfo[['MRN', 'Visit_id', 'Order_id', 'Order_time', 'description', 'antibiotic', 'susceptibility', 'organism']]
OISmall.drop_duplicates( subset = ('MRN', 'Visit_id'), inplace = True)

OISmall['counts']= OISmall.groupby('MRN')['Visit_id'].transform(pd.Series.count)


OISmall

/home/cuda/anaconda3/envs/py3/lib/python3.5/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/home/cuda/anaconda3/envs/py3/lib/python3.5/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,MRN,Visit_id,Order_id,Order_time,description,antibiotic,susceptibility,organism,counts
0,3525540,66008241995,103107206,2015-10-23 00:00:00.000,URINE CULTURE,Amikacin,Susceptible,ESCHERICHIA COLI,4
17,3525540,66009338937,113165749,2016-01-07 10:04:00.000,URINE CULTURE,Amikacin,Susceptible,ESCHERICHIA COLI,4
34,3525540,66009427034,113165754,2016-01-14 09:25:00.000,URINE CULTURE,Amikacin,Susceptible,ESCHERICHIA COLI,4
51,3525540,66009459910,113165760,2016-01-18 09:21:00.000,URINE CULTURE,Amikacin,Susceptible,ESCHERICHIA COLI,4
68,3527611,22257231,66201425,2013-07-19 14:09:00.000,"BODY SITE W/GRAM STAIN, AEROBIC ONLY",Cefazolin,Susceptible,"STAPHYLOCOCCUS, COAGULASE NEGATIVE",1
77,3527835,19655474,52542155,2012-09-08 12:45:00.000,FLUID CULTURE IN BLOOD BOTTLES,Amikacin,Susceptible,ESCHERICHIA COLI,1
101,3530607,17948957,43472716,2012-02-01 00:00:00.000,BLOOD CULTURE ROUTINE,Cefazolin,Susceptible,STAPHYLOCOCCUS AUREUS,1
124,3531696,22764652,68718916,2013-09-13 08:21:00.000,URINE CULTURE,Amikacin,Susceptible,ESCHERICHIA COLI,1
179,3531860,17484203,41020026,2011-11-27 00:00:00.000,"BODY SITE W/GRAM STAIN, AEROBIC ONLY",Amikacin,Susceptible,ESCHERICHIA COLI,8
225,3531860,18847426,48153037,2012-05-30 00:00:00.000,"BODY SITE W/GRAM STAIN, AEROBIC ONLY",Amikacin,Susceptible,ESCHERICHIA COLI,8


## For each anti, which bugs are suscep?

In [68]:
antiToStudy = 'Cefazolin'

antiDF = orderInfo[['description', 'antibiotic', 'susceptibility', 'organism']]


antiSuscepDF = antiDF[antiDF['susceptibility'] == 'Susceptible']

antiSuscepDF['suscCount'] = antiSuscepDF.groupby('organism')['organism'].transform(pd.Series.count)
antiSuscepDF.sort_values('suscCount', ascending = False, inplace = True)
antiSuscepDF.drop_duplicates(subset = ('organism', 'suscCount'), inplace = True)


antiSuscepDF[['organism', 'suscCount']]

/home/cuda/anaconda3/envs/py3/lib/python3.5/site-packages/ipykernel/__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/cuda/anaconda3/envs/py3/lib/python3.5/site-packages/ipykernel/__main__.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/cuda/anaconda3/envs/py3/lib/python3.5/site-packages/ipykernel/__main__.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,organism,suscCount
3,ESCHERICHIA COLI,21638
654294,STAPHYLOCOCCUS AUREUS,7993
104651,KLEBSIELLA PNEUMONIAE SS. PNEUMONIAE,5099
965715,PROTEUS MIRABILIS,2290
992700,KLEBSIELLA OXYTOCA,701
555504,"STAPHYLOCOCCUS, COAGULASE NEGATIVE",586
691054,CITROBACTER KOSERI,563
140191,STAPHYLOCOCCUS EPIDERMIDIS,506
1000933,STAPHYLOCOCCUS LUGDUNENSIS,193
362625,STAPHYLOCOCCUS HOMINIS,117
